## **1. Import các thư viện cần thiết**

In [1]:
import os   
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split

RANDOM_SEED = 1
tf.random.set_seed(RANDOM_SEED)

## **2. Chuẩn bị dữ liệu**

In [2]:
# Khai báo hàm Windowing (dùng để tạo các cặp X, y cho time series data)
def slicing_window(df, df_start_idx, df_end_idx, input_size, label_size, offset):
    features = [] # Khai báo list dùng để lưu trữ các X
    labels = [] # Khai báo list dùng để lưu trữ các y

    window_size = input_size + offset # Tính kích thước của window

    # Nếu df_end_idx = chỉ mục cuối cùng bảng dữ liệu, cần phải dời xuống 1 khoảng = window size 
    if df_end_idx == None:
        df_end_idx = len(df) - window_size 
    
    # Duyệt qua từng mẫu dữ liệu
    for idx in range(df_start_idx, df_end_idx):
        feature_end_idx = idx + input_size # Tính chỉ mục kết thúc của X
        label_start_idx = idx + window_size - label_size # Tính chỉ mục bắt đầu của y

        feature = df[idx:feature_end_idx] # Lấy X
        label = df[label_start_idx:(idx+window_size)] # Lấy y

        features.append(feature) 
        labels.append(label)

    # Chuyển list thành np.ndarrray
    features = np.expand_dims(np.array(features), -1) 
    labels = np.array(labels)

    return features, labels

In [3]:
univariate_df = pd.read_csv('temp.csv')
univariate_df = univariate_df['Temperature (C)']
univariate_df.head(7)

0    9.472222
1    9.355556
2    9.377778
3    8.288889
4    8.755556
5    9.222222
6    7.733333
Name: Temperature (C), dtype: float64

In [4]:
INPUT_SIZE = 6 # Dùng `input_size` giờ trước
LABEL_SIZE = 1 # Để dự đoán nhiệt độ trong `label_size` giờ
OFFSET = 1 # Vào khoảng `offset` giờ tiếp theo 
BATCH_SIZE = 32

In [5]:
dataset_length = len(univariate_df) # Số lượng mẫu dữ liệu trong DataFrame
TRAIN_SIZE = 0.7 # Tỉ lệ kích thước tập train
VAL_SIZE = 0.2 # Tỉ lệ kích thước tập val
# Đổi tỉ lệ thành chỉ mục
TRAIN_END_IDX = int(TRAIN_SIZE * dataset_length) 
VAL_END_IDX = int(VAL_SIZE * dataset_length) + TRAIN_END_IDX

# Khởi tạo X, y cho tập train
X_train, y_train = slicing_window(univariate_df, 
                                  df_start_idx=0,
                                  df_end_idx=TRAIN_END_IDX,
                                  input_size=INPUT_SIZE,
                                  label_size=LABEL_SIZE,
                                  offset=OFFSET)

# Khởi tạo X, y cho tập val
X_val, y_val = slicing_window(univariate_df, 
                                  df_start_idx=TRAIN_END_IDX,
                                  df_end_idx=VAL_END_IDX,
                                  input_size=INPUT_SIZE,
                                  label_size=LABEL_SIZE,
                                  offset=OFFSET)

# Khởi tạo X, y cho tập test
X_test, y_test = slicing_window(univariate_df, 
                                  df_start_idx=VAL_END_IDX,
                                  df_end_idx=None,
                                  input_size=INPUT_SIZE,
                                  label_size=LABEL_SIZE,
                                  offset=OFFSET)

In [6]:
print(X_train.shape)
print(y_train.shape)

(67517, 6, 1)
(67517, 1)


In [7]:
# Khởi tạo tf.data.Dataset
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(BATCH_SIZE)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH_SIZE)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)

In [8]:
# Cấu hình các tham số tối ưu cho việc đọc dữ liệu
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## **3. Xây dựng mô hình**

In [9]:
# Khai báo normalization layer
normalize_layer = tf.keras.layers.Normalization(axis=-1)
normalize_layer.adapt(np.vstack((X_train, X_val, X_test)))

In [12]:
# Xây dựng hàm khởi tạo model
def build_model(input_shape):
    model = tf.keras.Sequential([
        # Input layer (nhận vào 1 string)
        tf.keras.Input(shape=input_shape, name='input_layer'),         
        normalize_layer,
        tf.keras.layers.Flatten(),

        # Dense 1 
        tf.keras.layers.Dense(32, activation='relu'),      

        # Dense 2
        tf.keras.layers.Dense(32, activation='relu'),              
                       
                                
        # Output Layer
        tf.keras.layers.Dense(1, 
                              kernel_initializer=tf.initializers.GlorotUniform(seed=RANDOM_SEED),
                              name='output_layer') 
    ],
    name='rnn_model')

    return model

In [13]:
INPUT_SHAPE = X_train.shape[-2:] 
model = build_model(INPUT_SHAPE)
model.summary()

Model: "rnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  multiple                 3         
 n)                                                              
                                                                 
 flatten_1 (Flatten)         (None, 6)                 0         
                                                                 
 dense_2 (Dense)             (None, 32)                224       
                                                                 
 dense_3 (Dense)             (None, 32)                1056      
                                                                 
 output_layer (Dense)        (None, 1)                 33        
                                                                 
Total params: 1,316
Trainable params: 1,313
Non-trainable params: 3
_______________________________________________________

## **4. Cấu hình mô hình**

In [14]:
# Khai báo một số giá trị siêu tham số
EPOCHS = 20
LR = 1e-3

In [15]:
# Cấu hình một số thông tin cho mô hình
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR), # Sử dụng optimizer Adam
    loss=tf.keras.losses.MeanSquaredError(), # Sử dụng hàm loss Mean Squared Error
)

## **5. Huấn luyện mô hình**

In [16]:
history = model.fit(train_ds, 
                    validation_data=val_ds,
                    epochs=EPOCHS)

Epoch 1/20
2110/2110 [==============================] - 8s 3ms/step - loss: 27.6190 - val_loss: 2.0614
Epoch 2/20
2110/2110 [==============================] - 7s 3ms/step - loss: 2.5674 - val_loss: 1.6556
Epoch 3/20
2110/2110 [==============================] - 7s 3ms/step - loss: 2.3458 - val_loss: 1.6754
Epoch 4/20
2110/2110 [==============================] - 6s 3ms/step - loss: 2.2991 - val_loss: 1.6588
Epoch 5/20
2110/2110 [==============================] - 7s 3ms/step - loss: 2.2745 - val_loss: 1.6234
Epoch 6/20
2110/2110 [==============================] - 7s 3ms/step - loss: 2.2596 - val_loss: 1.6002
Epoch 7/20
2110/2110 [==============================] - 7s 3ms/step - loss: 2.2470 - val_loss: 1.5649
Epoch 8/20
2110/2110 [==============================] - 7s 3ms/step - loss: 2.2315 - val_loss: 1.5297
Epoch 9/20
2110/2110 [==============================] - 7s 3ms/step - loss: 2.2140 - val_loss: 1.5050
Epoch 10/20
2110/2110 [==============================] - 7s 3ms/step - loss: 2.19

## **6. Đánh giá và trực quan hóa**

In [17]:
# Đánh giá mô hình trên tập test
test_evaluation = model.evaluate(test_ds)

302/302 [==============================] - 1s 2ms/step - loss: 1.2169


In [ ]:
train_loss = history.history['loss']# Đọc thông tin loss trên tập train
val_loss = history.history['val_loss'] # Đọc thông tin loss trên tập val

plt.figure(figsize=(10, 5)) # Cài đặt kích thước khung ảnh

plt.subplot(1, 2, 1) # Khởi tạo khung ảnh cho training loss
plt.xlabel('Epochs') # Hiển thị tên trục hoành là 'Epochs'
plt.ylabel('Loss') # Hiển thị tên trục tung là 'Loss'
plt.title('Training loss') # Hiển thị title của khung ảnh hiện tại là 'Training Loss'
plt.plot(train_loss, color='red') # Vẽ đường giá trị loss trên tập train qua từng epoch (đường vẽ màu đỏ)

plt.subplot(1, 2, 2) # Khởi tạo khung ảnh cho val loss
plt.xlabel('Epochs') # Hiển thị tên trục hoành là 'Epochs'
plt.ylabel('Loss') # Hiển thị tên trục tung là 'Loss'
plt.title('Validation loss') # Hiển thị title của khung ảnh hiện tại là 'Validation loss'
plt.plot(val_loss, color='orange') # Vẽ đường giá trị loss trên tập val qua từng epoch (đường vẽ màu đỏ)

plt.show() # Hiển thị 2 khung ảnh nhỏ

In [ ]:
def plot_difference(y, pred):
    plt.figure(figsize=(20, 6))
    times = range(len(y))
    y_to_plot = y.flatten()
    pred_to_plot = pred.flatten()

    plt.plot(times, y_to_plot, color='steelblue', marker='o', label='True value')
    plt.plot(times, pred_to_plot, color='orangered', marker='X', label='Prediction')

    plt.title('Temperature in every hours')
    plt.xlabel('Hour')
    plt.ylabel('Temperature (C)')
    plt.legend()
    plt.show()

plot_difference(y_test[:100], model.predict(X_test[:100], verbose=0))